<a href="https://colab.research.google.com/github/cjfghk5697/osan_data_analysis/blob/main/simply_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[참고 자료](https://dacon.io/competitions/official/235745/codeshare/3013?page=1&dtype=recent)

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/cjfghk5697/osan_data_analysis/main/csv/Dacon%20dataset/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/cjfghk5697/osan_data_analysis/main/csv/Dacon%20dataset/test.csv')
submission = pd.read_csv('https://github.com/cjfghk5697/osan_data_analysis/raw/main/csv/Dacon%20dataset/sample_submission.csv')

In [3]:
train.columns = [
    '단지코드', '임대건물구분', '지역', '전용면적', '전용면적별세대수', '단지내주차면수', '등록차량수'
]

test.columns = [
    '단지코드', '임대건물구분', '지역', '전용면적', '전용면적별세대수', '단지내주차면수'
]

In [4]:
train.isna().sum()


단지코드        0
임대건물구분      0
지역          0
전용면적        0
전용면적별세대수    0
단지내주차면수     0
등록차량수       0
dtype: int64

In [5]:
test.isna().sum()


단지코드        0
임대건물구분      0
지역          0
전용면적        0
전용면적별세대수    0
단지내주차면수     0
dtype: int64

In [12]:
local_map={}
for i, loc in enumerate(train['지역'].unique()):
    local_map[loc]=i
#'경상북도': 0, '경상남도': 1, '대전광역시': 2, '경기도': 3, '전라북도': 4, '강원도': 5, '광주광역시': 6, 
# '충청남도': 7, '부산광역시': 8, '제주특별자치도': 9, '울산광역시': 10, '충청북도': 11, '전라남도': 12, '대구광역시': 13, 
# '서울특별시': 14, '세종특별자치시': 15

case_map={}
for i, case in enumerate(train['임대건물구분'].unique()):
    case_map[case] = i
#'아파트': 0, '상가': 1

supply_map={}


train['지역'] = train['지역'].map(local_map)
test['지역'] = test['지역'].map(local_map)

train['임대건물구분'] = train['임대건물구분'].map(case_map)
test['임대건물구분'] = test['임대건물구분'].map(case_map)

test['임대건물구분'] = test['임대건물구분'].astype(float)
train['임대건물구분'] = train['임대건물구분'].astype(float)

In [13]:
x_train = train.iloc[:, 1:-1]
y_train = train.iloc[:,-1]
x_test = test.iloc[:,1:]

In [ ]:
from sklearn.metrics import make_scorer
def rmse(p,a):
    difference = p-a
    squared = difference ** 2
    mean = squared.mean()
    score = np.sqrt(mean)
    return score
rmse_scorer = make_scorer(rmse, greater_is_better=False)
rmse_scorer

from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

from sklearn.model_selection import GridSearchCV
parameters = {
    'max_depth':[1,2,3,4,5,6,7],
    'random_state':[10,30,50,70,90],
}
GS = GridSearchCV(model, param_grid=parameters, cv=20, scoring=rmse_scorer)
GS.fit(x_train,y_train)

GS.best_params_

In [9]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

model_Lasso = make_pipeline(RobustScaler(), Lasso(alpha=0.1, random_state=20))
model_ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.1, l1_ratio=0.7, random_state=20))
model_GBoost = GradientBoostingRegressor(n_estimators=10000, learning_rate=0.05, max_depth=4,max_features='sqrt', min_samples_leaf=10,min_samples_split=5,loss='huber',random_state=90)

model_Lasso.fit(x_train, y_train)
Lasso_predictions=model_Lasso.predict(x_test)
train_Lasso=model_Lasso.predict(x_train)

model_ENet.fit(x_train, y_train)
ENet_predictions=model_ENet.predict(x_test)
train_ENet=model_ENet.predict(x_train)

model_GBoost.fit(x_train, y_train)
GBoost_predictions=model_GBoost.predict(x_test)
train_GBoost=model_GBoost.predict(x_train)

ValueError: ignored

In [ ]:
predictions=(Lasso_predictions + ENet_predictions + GBoost_predictions) / 3

submission['num'] = predictions
submission.to_csv('include_supply_type_3_model.csv', index=False)